In [1]:
import torch
import numpy as np
import time
import cv2
import os

In [2]:
### PyTorch

In [3]:
### Nested

In [4]:
def darken_blend_8_torch(base, active):
  return torch.where(torch.greater(base, active), active, base)

In [5]:
def color_burn_8_torch(base, active):
  return torch.where(torch.eq(active, 0.0), 255.0, 255.0 - (255.0 - base) / active)

In [6]:
def lighten_blend_8_torch(base, active):
  return torch.where(torch.less(base, active), active, base)

In [7]:
def color_dodge_8_torch(base, active):
  return torch.where(torch.eq(active, 255.0), 255.0, base / (255.0 - active))

In [8]:
def overlay_blend_8_torch(base, active):
  return torch.where(torch.greater_equal(base, 128.0), 2 * base + base - 2 * base * base / 255.0 - 128.0, 2 * base * base / 128.0)

In [9]:
def multiply_blend_8_torch(base, active):
  return base * active / 255.0

In [10]:
def linear_burn_8_torch(base, active):
  return base + active - 255.0

In [11]:
def screen_blend_8_torch(base, active):
  return base + active - base * active / 255.0

In [12]:
def linear_dodge_8_torch(base, active):
  return base + active

In [13]:
### Single

In [14]:
def normal_blend_f_torch(base, active, opacity):
  return opacity * active + (1-opacity)*base

In [15]:
def normal_blend_8_torch(base, active, opacity):
  return opacity * active + (255.0 - opacity) * base

In [16]:
### Runner

In [17]:
rng = np.random.default_rng(1)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [18]:
folder = "./data/"

img_files = [os.path.join(folder, f) for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]

bases = []
actives = []

for _file in img_files:
    img = cv2.imread(_file, cv2.IMREAD_GRAYSCALE)
    rnd = rng.random(img.shape, dtype = np.float32) * 255
    bases.append(img)
    actives.append(rnd)

In [30]:
def mat_runner(bases, actives, f):
    total_time = 0
    for i in range(len(bases)):
        b = torch.from_numpy(bases[i]).to(device)
        a = torch.from_numpy(actives[i]).to(device)
        start_time = time.perf_counter()
        f(b, a)
        end_time = time.perf_counter()
        del a
        del b
        total_time += (end_time - start_time) * 1000
    return total_time

def vec_runner(bases, actives, f):
    total_time = 0
    for i in range(len(bases)):
        base = bases[i].flatten()
        active = actives[i].flatten()
        b = torch.from_numpy(base).to(device)
        a = torch.from_numpy(active).to(device)
        opacity = torch.from_numpy(rng.random(1, dtype = np.float32)).to(device)
        start_time = time.perf_counter()
        f(b, a, opacity)
        end_time = time.perf_counter()
        del a
        del b
        total_time += (end_time - start_time) * 1000
    return total_time

def timer(bases, actives, f, runner):
    runs = 5
    times = []
    for _ in range(runs):
        times.append(runner(bases, actives, f))
    times = np.array(times)
    print(f"{np.average(times)}ms +/- {np.std(times)}ms")

In [20]:
timer(bases, actives, darken_blend_8_torch, mat_runner)

95.14127150177956ms +/- 9.552276584996ms


In [21]:
timer(bases, actives, color_burn_8_torch, mat_runner)

203.40639874339104ms +/- 9.176950285843265ms


In [22]:
timer(bases, actives, lighten_blend_8_torch, mat_runner)

91.59864517860115ms +/- 0.21304854932706857ms


In [23]:
timer(bases, actives, color_dodge_8_torch, mat_runner)

168.6593317426741ms +/- 1.4745537085521756ms


In [24]:
timer(bases, actives, overlay_blend_8_torch, mat_runner)

356.2803265172988ms +/- 2.394989602959726ms


In [25]:
timer(bases, actives, multiply_blend_8_torch, mat_runner)

64.6401193458587ms +/- 0.5352644389666419ms


In [26]:
timer(bases, actives, linear_burn_8_torch, mat_runner)

64.94646281935275ms +/- 0.30916351829087185ms


In [27]:
timer(bases, actives, screen_blend_8_torch, mat_runner)

116.02817266248167ms +/- 0.11179207061911214ms


In [28]:
timer(bases, actives, linear_dodge_8_torch, mat_runner)

31.87446901574731ms +/- 0.4331737883674408ms


In [31]:
timer(bases, actives, normal_blend_f_torch, vec_runner)

142.37591503188014ms +/- 1.3606784973507235ms


In [32]:
timer(bases, actives, normal_blend_8_torch, vec_runner)

142.80818467959762ms +/- 0.7610048509636361ms


: 